In [1]:
import pandas as pd

In [123]:
df = pd.read_csv('./data/인구밀도(2021-2022).csv', encoding='utf-8')
df = df[df['동별(3)'] == '소계']
df_population = df[['동별(2)', '2021', '2022']]
df_population.columns = ['자치구명', '인구_2021', '인구_2022']
df_population.drop(index=1, inplace=True)
df_population['인구_2021'] = df_population['인구_2021'].astype(int)
df_population['인구_2022'] = df_population['인구_2021'].astype(int)

/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/882128261.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_population.drop(index=1, inplace=True)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/882128261.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_population['인구_2021'] = df_population['인구_2021'].astype(int)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/882128261.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [124]:
df_elec = pd.read_csv('./data/ghgEmissions_data_electricity.csv', encoding='utf-8', skiprows=1)
df_elec = df_elec[['자치구명', '계']]
df_elec['계'] = df_elec['계'].str.replace(',', '').astype('float')
df_gas = pd.read_csv('./data/ghgEmissions_data_gas.csv', encoding='utf-8', skiprows=1)
df_gas = df_gas[['자치구명', '계']]
df_gas['계'] = df_gas['계'].str.replace(',', '').astype('float')
df_district_heating = pd.read_csv('./data/ghgEmissions_data_district_heating.csv', encoding='utf-8', skiprows=1)
df_district_heating = df_district_heating[['자치구명', '계']]
df_district_heating['계'] = df_district_heating['계'].str.replace(',', '').astype('float')
ghg_emissions = pd.merge(df_elec, df_gas, on='자치구명')
ghg_emissions = pd.merge(ghg_emissions, df_district_heating, on='자치구명')
ghg_emissions['온실가스배출량'] = ghg_emissions['계_x'] + ghg_emissions['계_y'] + ghg_emissions['계']
ghg_emissions.drop(['계_x', '계_y', '계'], axis=1, inplace=True)

In [125]:
env_index = pd.merge(df_population, ghg_emissions, on='자치구명')
env_index['온실가스(1인당)'] = env_index['온실가스배출량'] / env_index['인구_2022']
env_index.drop(['온실가스배출량'], axis=1, inplace=True)

In [126]:
df = pd.read_csv('./data/전력+사용량(용도별).csv', encoding='utf-8')
elec_consumption = df[['자치구별(2)', '2022']]
elec_consumption.drop(index=range(4), inplace=True)
elec_consumption.columns = ['자치구명', '전력사용량']

/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/1242192353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_consumption.drop(index=range(4), inplace=True)


In [127]:
env_index = pd.merge(env_index, elec_consumption, on='자치구명')

In [128]:
df = pd.read_csv('./data/서울특별시 공공자전거 대여소별 이용정보(월별)_23.1-6.csv', encoding='cp949')
df['대여건수'] = pd.to_numeric(df['대여건수'].str.replace(',', ''), errors='coerce')
df['반납건수'] = pd.to_numeric(df['반납건수'].str.replace(',', ''), errors='coerce')
bicycle_usage = df.groupby('자치구', as_index=False).agg({'대여건수': 'sum', '반납건수': 'sum'})
bicycle_usage.columns = ['자치구명', '자전거_대여건수', '자전거_반납건수']
bicycle_usage['공공자전거_이용건수'] = (bicycle_usage['자전거_대여건수'] + bicycle_usage['자전거_반납건수'])/2
bicycle_usage['공공자전거_이용건수'] = bicycle_usage['공공자전거_이용건수'].astype(int)
bicycle_usage.drop(['자전거_대여건수', '자전거_반납건수'], axis=1, inplace=True)

In [129]:
env_index = pd.merge(env_index, bicycle_usage, on='자치구명')

In [130]:
df = pd.read_csv('./data/급수+사용량+및+사용료(2012년+이후).csv', encoding='utf-8')
water_usage = df[['자치구별(2)', '2022']]
water_usage.columns = ['자치구명', '급수사용량']
water_usage.drop(index=range(4), inplace=True)
water_usage['급수사용량'] = water_usage['급수사용량'].astype('float')

/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/19716127.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_usage.drop(index=range(4), inplace=True)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/19716127.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_usage['급수사용량'] = water_usage['급수사용량'].astype('float')


In [131]:
env_index = pd.merge(env_index, water_usage, on='자치구명')

In [104]:
env_index['급수사용량(1인당)'] = env_index['급수사용량'] / env_index['인구_2022']
env_index.drop(['급수사용량'], axis=1, inplace=True)

In [132]:
waste_emission = pd.read_csv('./data/주민+1인당+생활폐기물(쓰레기)+배출량.csv', encoding='utf-8')
waste_emission.columns = ['자치구명', '쓰레기배출량(1인당)', '생활폐기물 배출량', '주민수']
waste_emission = waste_emission[['자치구명', '쓰레기배출량(1인당)']]
waste_emission.drop(range(2), inplace=True)

In [133]:
env_index = pd.merge(env_index, waste_emission, on='자치구명')

In [134]:
df = pd.read_csv('./data/대기오염(구별).csv', encoding='utf-8')
air_data = df[['구분별(1)']].join(df.filter(like='2022', axis=1))
air_data.drop(index=range(1,4), inplace=True)
air_data.columns = ['자치구명', '아황산가스', '일산화탄소', '이산화질소', '미세먼지', '초미세먼지', '오존']
air_data.drop(index=0, inplace=True)

In [135]:
env_index = pd.merge(env_index, air_data, on='자치구명')

In [136]:
df = pd.read_csv('./data/서울시_공원통계(2022).csv', encoding='utf-8')
park = df[['자치구별(2)', '2022.2']]
park.drop(index=range(5), inplace=True)
park.drop(index=30, inplace=True)
park.columns = ['자치구명', '공원면적']
park['공원면적'] = park['공원면적'].astype('float')

/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/2230141381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park.drop(index=range(5), inplace=True)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/2230141381.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park.drop(index=30, inplace=True)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_31870/2230141381.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [137]:
env_index = pd.merge(env_index, park, on='자치구명')

In [138]:
env_index['공원면적(1인당)'] = env_index['공원면적'] / env_index['인구_2022']
env_index.drop(columns=['공원면적', '인구_2021', '인구_2022'], inplace=True)

In [139]:
env_index

,자치구명,온실가스(1인당),전력사용량,공공자전거_이용건수,급수사용량,쓰레기배출량(1인당),아황산가스,일산화탄소,이산화질소,미세먼지,초미세먼지,오존,공원면적(1인당)
0,종로구,6.764409,1721240,726636,25466.0,1.49,0.003,0.5,0.021,32,19,0.031,0.074353
1,중구,10.138907,2305200,500302,27276.0,2.26,0.003,0.4,0.024,30,19,0.030,0.024329
2,용산구,3.827852,1514990,441899,27382.0,1.24,0.003,0.4,0.021,33,19,0.030,0.007476
3,성동구,4.072864,2235050,859819,31437.0,0.93,0.003,0.4,0.021,35,16,0.028,0.010537
4,광진구,2.736378,1616001,1042726,35948.0,0.71,0.003,0.5,0.019,31,18,0.027,0.009812
5,동대문구,2.676331,1457689,777649,35364.0,0.91,0.003,0.4,0.022,34,19,0.032,0.003453
6,중랑구,2.238323,1161372,628434,64523.0,0.74,0.003,0.4,0.021,33,18,0.031,0.013343
7,성북구,2.462184,1497475,529069,41041.0,0.85,0.003,0.5,0.023,33,18,0.028,0.019373
8,강북구,2.381970,933057,331973,27429.0,0.85,0.002,0.4,0.015,34,18,0.032,0.047909
9,도봉구,2.313772,894641,451406,28526.0,0.79,0.003,0.5,0.016,30,17,0.031,0.031784


In [140]:
env_index.to_csv('./result/seoul_env_index.csv', encoding='utf-8-sig')